In [ ]:
import nilearn.plotting as plotting
import pandas as pd

from glob import glob
from IPython.display import display
from os.path import join

import tbi

In [ ]:
output = '/data/illustration_workflow_output'
mni_file = 'illustration_data/MNI152_T1_1mm_brain.nii.gz'
atlas_file = 'illustration_data/New_atlas_cort_asym_sub.nii.gz'
template_file = 'illustration_data/T_template0.nii.gz'
dcmfiles = 'illustration_data/dcmfiles/*'

In [ ]:
convert_dir = join(output, 'convert')
tbi.convert(dcmfiles, convert_dir, use_dcm2niix=True)

In [ ]:
nii_files = glob(join(convert_dir, "*.nii.gz"))

for nii_file in nii_files:
    print('Plotting {0}'.format(nii_file))
    plotting.plot_img(nii_file)
    plotting.show()

In [ ]:
preprocessing_dir = join(output, 'preprocessing')
tbi.preprocessing(join(convert_dir, '*.nii.gz'), 
                  preprocessing_dir,
                  mni_file,)

In [ ]:
nii_files = glob(join(preprocessing_dir, "*.nii.gz"))

for nii_file in nii_files:
    print('Plotting {0}'.format(nii_file))
    plotting.plot_img(nii_file)
    plotting.show()

In [ ]:
skull_strip_dir = join(output, 'brains')
tbi.skull_strip(join(preprocessing_dir, '*_normalizedWarped.nii.gz'),
                skull_strip_dir)

In [ ]:
nii_files = glob(join(skull_strip_dir, "*.nii.gz"))

for nii_file in nii_files:
    print('Plotting {0}'.format(nii_file))
    plotting.plot_img(nii_file)
    plotting.show()

In [ ]:
registration_dir = join(output, 'registration')
tbi.registration(join(skull_strip_dir, '*.nii.gz'), 
                 registration_dir, 
                 template_file)

In [ ]:
nii_files = glob(join(registration_dir, '*/*.nii.gz'))

for nii_file in nii_files:
    print(nii_file)
    img = tbi.load_as_3d(nii_file)
    plotting.plot_img(img)
    plotting.show()

In [ ]:
segmentation_dir = join(output, 'segmentation')
tbi.segmentation(join(registration_dir, '*/*.nii.gz'), 
                 segmentation_dir, 
                 atlas_file)

In [ ]:
nii_files = glob(join(segmentation_dir, '*/*.nii.gz'))
for nii_file in nii_files:
    print(nii_file)
    plotting.plot_img(nii_file)
    plotting.show()

In [ ]:
label_geometry_measures_dir = join(output, 'label_geometry_measures')
tbi.label_geometry_measures(join(segmentation_dir, '*/*.nii.gz'), 
                            label_geometry_measures_dir)

In [ ]:
csv_files = glob(join(label_geometry_measures_dir, "*.csv"))

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    display(df.iloc[0:115,0:5])

In [ ]:
image_intensity_stat_dir = join(output, 'image_intensity_stat')
tbi.image_intensity_stat(join(registration_dir, 'Affine2SyN/*affine2Syn1Warp.nii.gz'),
                        image_intensity_stat_dir,
                        atlas_file)

In [ ]:
csv_files = glob(join(image_intensity_stat_dir, "*.csv"))

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    display(df.iloc[0:115,0:5])
